In [1]:
import pandas as pd

In [2]:
pd.read_csv('dist_Ind.csv')

,index,Universidad,Facultad,coords_o,coords_d,distance_km,ranking
0,888,Instituto Universitario de Seguridad Marítima,Departamento Académico Escuela de Oficiales,"(-34.579419, -58.532869999999996)","(-34.0940568, -59.0199532)",70.054725,24.5
1,888,Instituto Universitario de Seguridad Marítima,Departamento Académico Escuela de Suboficiales,"(-34.579419, -58.532869999999996)","(-34.0940568, -59.0199532)",70.054725,24.5
2,888,Universidad de Buenos Aires,Facultad de Ciencias Exactas y Naturales,"(-34.579419, -58.532869999999996)","(-34.54684, -58.43995)",9.261583,3.0
3,888,Universidad de Buenos Aires,Facultad de Farmacia y Bioquímica,"(-34.579419, -58.532869999999996)","(-34.597903, -58.397458400000005)",12.591102,7.0
4,888,Universidad de Buenos Aires,Facultad de Ingeniería,"(-34.579419, -58.532869999999996)","(-34.68477, -58.44631)",14.127400,8.5
5,888,Universidad Nacional Arturo Jauretche,Instituto de Ingeniería y Agronomía,"(-34.579419, -58.532869999999996)","(-34.796310600000005, -58.27863860000001)",33.490756,19.0
6,888,Universidad Nacional de Avellaneda,Departamento de Tecnología y Administración,"(-34.579419, -58.532869999999996)","(-34.6628049, -58.372224800000005)",17.395635,11.0
7,888,Universidad Nacional de General Sarmiento,Instituto de Industria,"(-34.579419, -58.532869999999996)","(-34.5208625, -58.7003265)",16.686282,10.0
8,888,Universidad Nacional de Hurlingham,Instituto de Biotecnología,"(-34.579419, -58.532869999999996)","(-34.6164566, -58.6342231)",10.164765,4.0
9,888,Universidad Nacional de Lanús,Departamento de Desarrollo Productivo y Tecnol...,"(-34.579419, -58.532869999999996)","(-34.7342286, -58.3901502)",21.589232,13.0
